In [1]:
from langgraph.graph import StateGraph, END, START
from typing import TypedDict
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

In [3]:
load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [4]:
class FeedbackSchema(BaseModel):

    feedback: str = Field(description="Feedback of the essay")
    score: int = Field(description="Give a score 1 to 10",ge=0,lt=11)

In [5]:
structured_llm = llm.with_structured_output(FeedbackSchema)